# 신경망 클래스

In [38]:
import numpy as np
import scipy
from scipy import special
import matplotlib.pyplot as plt

In [60]:
#신경망 클래스의 정의
class neuralNetwork2:
    
    #신경망 초기화 하기
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate):
        #입력, 은닉, 출력 계층의 노드 개수 설정
        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        #가중치 행렬 wih와 who
        #배열 내 가중치는 w_i_j로 표기, 노드i에서 다음 계층의 노드j로 연결됨을 의미
        self.wih1 = (np.random.rand(self.hnodes1, self.inodes) - 0.5)
        self.wh1h2 = (np.random.rand(self.hnodes2, self.hnodes1) - 0.5)
        self.wh2o = (np.random.rand(self.onodes, self.hnodes2) - 0.5)
        
        #학습률
        self.lr = learningrate
        
        #활성화 함수로 시그모이드 함수를 이용s
        self.activation_function = lambda x: scipy.special.expit(x)
        pass
    
    #신경망 학습시키기
    def train(self, input_list, target_list):
        #입력리스트들
        inputs = np.array(input_list, ndmin=2).T
        targets = np.array(target_list, ndmin=2).T
    
        #계층간 전달
        hidden_inputs1 = np.dot(self.wih1, inputs)
        hidden_outputs1 = self.activation_function(hidden_inputs1)
        hidden_inputs2 = np.dot(self.wh1h2, hidden_outputs1)
        hidden_outputs2 = self.activation_function(hidden_inputs2)
        final_inputs = np.dot(self.wh2o, hidden_outputs2)
        final_outputs  = self.activation_function(final_inputs)
        
        # (실제 값 - 계산값) 오차
        output_errors = targets - final_outputs
        hidden_errors2 = np.dot(self.wh2o.T, output_errors)
        hidden_errors1 = np.dot(self.wh1h2.T, hidden_errors2)
        
        self.wh2o += self.lr * np.dot((output_errors * final_outputs * (1.0 -final_outputs)), np.transpose(hidden_outputs2))
        self.wh1h2 += self.lr * np.dot((hidden_errors2 * hidden_outputs2 * (1.0 -hidden_outputs2)), np.transpose(hidden_outputs1))
        self.wih1 += self.lr * np.dot((hidden_errors1 * hidden_outputs1 * (1.0 -hidden_outputs1)), np.transpose(inputs))
        pass
    
    #신경망에 질의하기
    def query(self, input_list):
        inputs = np.array(input_list, ndmin=2).T
    
        hidden_inputs1 = np.dot(self.wih1, inputs)
        hidden_outputs1 = self.activation_function(hidden_inputs1)
        hidden_inputs2 = np.dot(self.wh1h2, hidden_outputs1)
        hidden_outputs2 = self.activation_function(hidden_inputs2)
        final_inputs = np.dot(self.wh2o, hidden_outputs2)
        final_outputs  = self.activation_function(final_inputs)
        return  final_outputs

# 신경망 테스트 클래스

In [61]:
def testNeuralNet(e):
    test_data_file = open("mnist_dataset/mnist_test.csv","r")
    test_data_list = test_data_file.readlines()
    test_data_file.close()

    #신경망의 성능의 지표가 되는 성적표를 아무 값도 가지지 않도록 초기화
    scorecard = []

    #테스트 데이터의 모음 내의 모든 레코드 탐색
    for record in test_data_list:
        #레코드를 쉼표에 의해 분리
        all_values = record.split(',')
        correct_label = int(all_values[0])
        #print(correct_label, "correct label")
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        outputs = n.query(inputs)
        label = np.argmax(outputs)

        #print(label, "network's asnwer")

        if(label == correct_label):
            scorecard.append(1)
        else:
            scorecard.append(0)
            pass
        pass
    
    scorecard_array = np.asarray(scorecard)
    print("epochs = ", e,
     "hidden_nodes1 = ", hidden_nodes1, " hidden_nodes2 = ", hidden_nodes2,
     "learning_rate = ", learning_rate, "performance = ",scorecard_array.sum() / scorecard_array.size)

# 하이퍼파라미터

In [62]:
input_nodes = 784
hidden_nodes1 = 300
hidden_nodes2 = 100
output_nodes = 10
learning_rate = 0.005
epochs = 5

n = neuralNetwork2(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

#100~200
#0.3~0.03
#5~10

# 신경망 학습 및 테스트

In [ ]:
train_data_file = open("mnist_dataset/mnist_train.csv","r")
train_data_list = train_data_file.readlines()
train_data_file.close()

for e in range(epochs):
    for record in train_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01 #0~255값 -> 0.01~1사이의 값
        targets = np.zeros(output_nodes) + 0.01 #결과값중 아닌것들은 0.01
        targets[int(all_values[0])] = 0.99      #결과값중 맞는것은 0.99
        n.train(inputs, targets)
    testNeuralNet(e+1)
   
pass

# 201601769 나요한

1. 같은 에폭 가정 ->  hidden_node수 조정
2. 같은 hidden_node일때 -> learning_rate 조정
3. 마지막으로 epoch수 조정

epochs =  2 hidden_nodes1 =  300  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.854
epochs =  2 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8625
epochs =  2 hidden_nodes1 =  200  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8554
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.2 performance =  0.8762

epochs =  2 hidden_nodes1 =  50  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8417
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.8162
epochs =  2 hidden_nodes1 =  200  hidden_nodes2 =  300 learning_rate =  0.2 performance =  0.821

In [ ]:
epochs =  1 hidden_nodes1 =  100  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.809
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.8474
epochs =  3 hidden_nodes1 =  100  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.8403
epochs =  4 hidden_nodes1 =  100  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.8295
epochs =  5 hidden_nodes1 =  100  hidden_nodes2 =  200 learning_rate =  0.2 performance =  0.8477

In [ ]:
epochs =  1 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.2 performance =  0.8436
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.2 performance =  0.8727
epochs =  3 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.2 performance =  0.8373
epochs =  4 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.2 performance =  0.8505
epochs =  5 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.2 performance =  0.8276

In [ ]:
epochs =  1 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8712
epochs =  2 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8465
epochs =  3 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8247
epochs =  4 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8365
epochs =  5 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.2 performance =  0.8218

In [ ]:
epochs =  1 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.05 performance =  0.934
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.05 performance =  0.9503
epochs =  3 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.05 performance =  0.9529
epochs =  4 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.05 performance =  0.9508
epochs =  5 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.05 performance =  0.9453

In [ ]:
epochs =  1 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.025 performance =  0.9426
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.025 performance =  0.9581
epochs =  3 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.025 performance =  0.9603
epochs =  4 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.025 performance =  0.964
epochs =  5 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.025 performance =  0.9656

In [ ]:
epochs =  1 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.01 performance =  0.9283
epochs =  2 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.01 performance =  0.9461
epochs =  3 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.01 performance =  0.9566
epochs =  4 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.01 performance =  0.963
epochs =  5 hidden_nodes1 =  100  hidden_nodes2 =  50 learning_rate =  0.01 performance =  0.9675

In [ ]:
epochs =  1 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.1 performance =  0.9342
epochs =  2 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.1 performance =  0.9417
epochs =  3 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.1 performance =  0.9194
epochs =  4 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.1 performance =  0.919
epochs =  5 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.1 performance =  0.9265

In [ ]:
epochs =  1 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.01 performance =  0.9422
epochs =  2 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.01 performance =  0.9611
epochs =  3 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.01 performance =  0.9686
epochs =  4 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.01 performance =  0.9729
epochs =  5 hidden_nodes1 =  300  hidden_nodes2 =  100 learning_rate =  0.01 performance =  0.9752